# データ分析とモデル作成sample

## 内容

- タイタニックのデータを使用して，データ分析とモデル作成を行う

# Load modules

ライブラリ読み込み

In [ ]:
# ライブラリ読み込み
import sys, os
import time
import gc
from datetime import datetime as dt
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import japanize_matplotlib

import re # 正規表現

import seaborn as sns

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# 評価関数
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

%matplotlib inline

In [ ]:
gc.collect()

# Configuration

## warningの非表示

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 左寄せにするマジックコマンド

In [ ]:
%%html
<style>
    table{float:left}
    .MathJax{float: left;}
</style>

## データフレームの表示設定

In [ ]:
# データフレームの表示行数、表示列数
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

# カラム内の文字数。デフォルトは50。
pd.set_option('display.max_colwidth', 100)

# Constants

## paths

In [ ]:
input_dpath = '../input/'
output_dpath = '../output/'

# Functions

## 数値計算

### 数字を四捨五入で丸める

In [ ]:
def pro_round(num, ndigits=0):
    """
    数字を四捨五入で丸める。

    Args:
        num: int or float
            丸めたい数字。

        ndigits: int, optional(default=0)
            丸めた後の小数部分の桁数。

    Returns:
        rounded: int or float
            丸めた後の数字。
    """
    num *= 10 ** ndigits
    rounded = ( 2* num + 1 ) // 2
    rounded /= 10 ** ndigits

    if ndigits == 0:
        rounded = int(rounded)

    return rounded

### スタージェスの公式

In [ ]:
def sturges_rule(num):
    """
    スタージェスの公式を用いて、
    サンプルサイズから階級(カテゴリ、ビン(bins))の数を計算する。
    公式のTeX表記: \[bins = 1 + \log_2{N} \nonumber \]

    Args:
        num: int
            サンプルサイズ。原則1以上の整数を想定。
    
    Returns:
        n_bins: int
            スタージェスの公式から導かれた適切な階級の数。
    """
    # numが0以下の時は1を返す
    if num <= 0:
        num = 1
        return 1
    
    # スタージェスの公式
    n_bins = int(pro_round(1 + np.log2(num), 0))
    
    return n_bins

# Load data

## データ準備

In [ ]:
# タイタニックデータ
# seabornからロード
titanic_df = sns.load_dataset('titanic')

# Explatory data analysis

## 基本的な情報を確認

In [ ]:
# 上から3行読む
titanic_df.head(3)

In [ ]:
# head()の中を何も指定しないと5行読み込む扱い
titanic_df.head()

In [ ]:
# tail()は最後の5行
titanic_df.tail()

In [ ]:
# (行数, 列数)を確認
titanic_df.shape

## 基本統計量を見る

In [ ]:
# 基本統計量を見る
titanic_df.describe()

## 相関を見る

In [ ]:
corr = titanic_df.corr()
corr

In [ ]:
# ヒートマップで表示も可能
sns.heatmap(corr)

### 相関係数(の絶対値)が高い順に並べる

In [ ]:
# 目的変数
target = 'survived'

In [ ]:
# 相関係数の絶対値が高い順に並べる
corr0 = corr[[target]]
corr0['abs_corr'] = corr0[target].abs()
corr0 = corr0.sort_values(by='abs_corr', ascending=False)
corr0

以下2つは同じ
titanic_df['pclass']
titanic_df.pclass

## 度数を見てみる

### pclass

In [ ]:
titanic_df.pclass.value_counts()

### 性別

In [ ]:
titanic_df.sex.value_counts()

## 各変数の尺度を確認することが重要

参考: http://www.gen-info.osaka-u.ac.jp/MEPHAS/express/express0.html  
  
- 順序尺度以上：順序尺度・間隔尺度・比例尺度のいずれかである場合です  
- 間隔尺度以上：間隔尺度・比例尺度のいずれかである場合です  
  
### 名義尺度
単に区別するために用いられている尺度。  
例えば、血液型でＡ型・Ｂ型・Ｏ型・ＡＢ型を、 それぞれ０・１・２・３と数値に対応させたもの。  
これらの変数の平均値を求めてもまったく意味がありません。  
  
### 順序尺度
大小関係にのみ意味がある尺度。  
例えば、治療効果の判定において、 悪化・不変・改善・著効を、それぞれ-１・０・１・２と数値に対応させたもの。  
平均値は定義できないが中央値は定義できます。  
  
### 間隔尺度
数値の差のみに意味がある尺度。  
「距離尺度」とも呼びます。順序尺度の性質も備えています。  
例えば、温度が10℃から15℃になったときに、50％の温度上昇があったとはいいません。 温度が10℃から15℃になったときも、100℃から105℃になったときも、 ともに５℃の温度上昇です。そして、５℃という数値には意味があります。  
  
### 比例尺度
数値の差とともに数値の比にも意味がある尺度。  
「比尺度」とも呼びます。順序尺度・間隔尺度の性質も備えています。  
例えば、体重は50kgから60kgになったときと、100kgから110kgになったときとは、 同じ10kgの増加であっても、前者は20％増、後者は10％増です。 また、比が定義できるということは絶対零点を持つことと同じことを表します。  

## Excelで保存してみる

to_excel(ファイル名, index=(indexをつけるかどうか))

In [ ]:
titanic_fpath = f'{input_dpath}titanic_20220910.xlsx'

In [ ]:
# 保存する
titanic_df.to_excel(titanic_fpath, index=False)

## Excelを読み込む

In [ ]:
# 読み込む
nakami_df = pd.read_excel(titanic_fpath)
nakami_df.head(3)

# Preprocessing

In [ ]:
df = titanic_df.copy()
df.head(3)

In [ ]:
df.shape

## leakage対策

leakageについて  
https://www.datarobot.com/jp/wiki/target-leakage/

### aliveがsurvivedの情報を含んでいるのでaliveカラムを削除

In [ ]:
# 重複した行を削除
df[[target, 'alive']].drop_duplicates()

In [ ]:
df = df.drop('alive', axis=1)
df.head(3)

## 欠損処理

### 今回は簡易的に考えるため欠損は削除する

In [ ]:
df.shape

In [ ]:
df = df.dropna(
    axis=0,
    subset=[
        'survived',
        'pclass',
        'age',
        'sibsp',
        'parch',
        'fare',
        'class',
        'who',
        'adult_male'
    ]
)

In [ ]:
df.shape

## 型を修正

In [ ]:
df.columns.tolist()

In [ ]:
int_cols = [
    'pclass',
    'age',
    'sibsp',
    'parch'
]

In [ ]:
for col in int_cols:
    
    # int型にする
    df[col] = df[col].astype(int)

## 階級に分ける

### 階級に分けるカラムの指定

In [ ]:
cls_cols = ['age', 'fare']

### 階級数をスタージェスの公式で計算

In [ ]:
n_class = sturges_rule(len(df))
n_class

### 階級に分ける場合

- 参考: [pandasのcut, qcut関数でビニング処理（ビン分割）](https://note.nkmk.me/python-pandas-cut-qcut-binning/)

In [ ]:
df['age'].value_counts().head()

In [ ]:
# ユニーク数の確認
df['age'].nunique()

In [ ]:
# 階級に分ける(ビニング処理)
df['age_splitted'] = pd.cut(df['age'], bins=n_class)

In [ ]:
df['age_splitted'].value_counts().head()

In [ ]:
# ユニーク数の確認
df['age_splitted'].nunique()

In [ ]:
df['fare'].value_counts().head()

In [ ]:
# ユニーク数の確認
df['fare'].nunique()

In [ ]:
# 階級に分ける(ビニング処理)
df['fare_splitted'] = pd.cut(df['fare'], bins=n_class)

In [ ]:
df['fare_splitted'].value_counts().head()

In [ ]:
# ユニーク数の確認
df['fare_splitted'].nunique()

## ダミー変数化

### カテゴリ変数のカラムの指定

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
# カテゴリ変数のカラム
cate_cols = df.columns.tolist()
cate_cols = list(set(cate_cols) - {target, 'age', 'fare'})
cate_cols

### ダミー変数化

In [ ]:
df.columns.tolist()

In [ ]:
df = pd.get_dummies(df, dummy_na=True, columns=cate_cols)
df.head(3)

In [ ]:
df.shape

In [ ]:
# カラムの確認
df.columns.tolist()

## 目的変数と説明変数で分ける

In [ ]:
df.columns.tolist()

In [ ]:
data_x = df.drop(labels=[target], axis=1)
data_y = df[target]

## trainとtestに分割

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(
    data_x,
    data_y,
    random_state=57,
    test_size=0.3
)

In [ ]:
train_x.head()

In [ ]:
print('train_x.shape:', train_x.shape)
print('test_x.shape:', test_x.shape)
print('train_y.shape:', train_y.shape)
print('test_y.shape:', test_y.shape)

# RandomForest

## create model

In [ ]:
rfc = RandomForestClassifier(
    bootstrap=True,
    criterion='gini',
    max_depth=11,
    min_samples_leaf=5,
    n_estimators=100,
    random_state=57
)
rfc.fit(train_x, train_y)

## 寄与度分析

In [ ]:
# 説明変数
features = train_x.columns

# 寄与率
importances = rfc.feature_importances_

In [ ]:
# importances描き方
importance_df = pd.DataFrame(
    {'feature': features, 'importance':  importances}
).sort_values(by='importance', ascending=True)

importance_df.tail(10).plot(
    x='feature',
    y='importance',
    kind='barh',
    figsize=(12,9)
)

display(importance_df.sort_values(by='importance', ascending=False).head(10))

## 木の一つを可視化する

In [ ]:
# 試しに木の一つを視覚化する
estimators = rfc.estimators_
len(estimators)

In [ ]:
# 可視化の設定
plt.figure(
    figsize=(10, 5), # 描画サイズ
    facecolor='white', # 背景色
    dpi=200 # 解像度
)

# 決定木の可視化
tree.plot_tree(
    decision_tree=estimators[0], # 決定木モデル
    max_depth=3, # 表示する木の深さ
    feature_names=features, # 説明変数名
    class_names=['死亡', '生存'], # 目的変数の各クラス名
    fontsize=4,
    filled=True # ノードに色を付ける
)